<font style='font-size:1.5em'>**Initial Data Analysis**</font>

<font style='font-size:1.2em'>Using Reddit CSV</font>

**Author: Sentipedia**  

**Course: DS105L Project** 

**DATE: 3/29/2023** 

---


## Imports 

### Importing Libraries 

In [6]:
# importing libraries for web scrapping and API acess
import requests
from bs4 import BeautifulSoup

# importing libraries for dataframes 
import pandas as pd
import numpy as np

# importing libraries for plots 
from plotnine import *

# importing libraries for ploting networks
import networkx as nx
import matplotlib.pyplot as plt

# Data

### Looking at the Data 

In [ ]:
# import language model 
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# downloading NLTK analyzers 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [8]:
df_reddit = pd.read_csv('reddit_full.csv', usecols=['url', 'comment', 'comment_id'])
df_reddit.head()

,url,comment,comment_id
0,https://www.reddit.com/r/unitedkingdom/comment...,"Agree. With the Tories pushing Brexit, they h...",1_2_1_4_1
1,https://www.reddit.com/r/unitedkingdom/comment...,I just thought that with plenty of vaccination...,1_8_1_1
2,https://www.reddit.com/r/unitedkingdom/comment...,"If you don't listen to your core voter base, t...",1_11_1_1
3,https://www.reddit.com/r/unitedkingdom/comment...,Much as I agree with your potential projection...,1_15
4,https://www.reddit.com/r/unitedkingdom/comment...,"Brexit, the word, is why it succeeded. Nobody ...",2


# Using NLTK

In [54]:
df_reddit_nltk = df_reddit.copy()

def text_preprocess(comment):
    tokens = [word.lower() for word in word_tokenize(comment)]

    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    return lemmatized_tokens

def get_score_nltk(tokens):
    preprocessed_text = ' '.join(tokens)
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(preprocessed_text)
    return sentiment_scores

In [55]:
df_reddit_nltk['tokens'] = df_reddit_nltk['comment'].apply(text_preprocess)
df_reddit_nltk['score'] = df_reddit_nltk['tokens'].apply(get_score_nltk)

In [56]:
df_reddit_nltk

,url,comment,comment_id,tokens,score
0,https://www.reddit.com/r/unitedkingdom/comment...,"Agree. With the Tories pushing Brexit, they h...",1_2_1_4_1,"[agree, ., tory, pushing, brexit, ,, become, r...","{'neg': 0.317, 'neu': 0.452, 'pos': 0.231, 'co..."
1,https://www.reddit.com/r/unitedkingdom/comment...,I just thought that with plenty of vaccination...,1_8_1_1,"[thought, plenty, vaccination, coming, brexit,...","{'neg': 0.079, 'neu': 0.452, 'pos': 0.469, 'co..."
2,https://www.reddit.com/r/unitedkingdom/comment...,"If you don't listen to your core voter base, t...",1_11_1_1,"[n't, listen, core, voter, base, ,, wo, n't, v...","{'neg': 0.04, 'neu': 0.889, 'pos': 0.07, 'comp..."
3,https://www.reddit.com/r/unitedkingdom/comment...,Much as I agree with your potential projection...,1_15,"[much, agree, potential, projection, ,, overwh...","{'neg': 0.114, 'neu': 0.768, 'pos': 0.118, 'co..."
4,https://www.reddit.com/r/unitedkingdom/comment...,"Brexit, the word, is why it succeeded. Nobody ...",2,"[brexit, ,, word, ,, succeeded, ., nobody, wou...","{'neg': 0.0, 'neu': 0.62, 'pos': 0.38, 'compou..."
...,...,...,...,...,...
11709,https://www.reddit.com/r/unitedkingdom/comment...,"By Britain they don't mean ordinary people, mo...",20_1_1,"[britain, n't, mean, ordinary, people, ,, n't,...","{'neg': 0.061, 'neu': 0.779, 'pos': 0.16, 'com..."
11710,https://www.reddit.com/r/unitedkingdom/comment...,Because the people on the leave side see this ...,22,"[people, leave, side, see, establishment, find...","{'neg': 0.12, 'neu': 0.783, 'pos': 0.097, 'com..."
11711,https://www.reddit.com/r/unitedkingdom/comment...,Because ultimately it is the houses of parliam...,24,"[ultimately, house, parliament, voted, brexit,...","{'neg': 0.19, 'neu': 0.701, 'pos': 0.109, 'com..."
11712,https://www.reddit.com/r/unitedkingdom/comment...,"Democracy changes laws and governments, but po...",35,"[democracy, change, law, government, ,, politi...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [47]:
tokens = df_reddit_nltk['tokens'][4]
preprocessed_text = ' '.join(tokens)
analyzer = SentimentIntensityAnalyzer()
sentiment_scores = analyzer.polarity_scores(preprocessed_text)

sentiment_scores

{'neg': 0.0, 'neu': 0.62, 'pos': 0.38, 'compound': 0.8316}

In [41]:
df_reddit_nltk

,url,comment,comment_id,tokens,score
0,https://www.reddit.com/r/unitedkingdom/comment...,"Agree. With the Tories pushing Brexit, they h...",1_2_1_4_1,"[agree, ., tory, pushing, brexit, ,, become, r...",None
1,https://www.reddit.com/r/unitedkingdom/comment...,I just thought that with plenty of vaccination...,1_8_1_1,"[thought, plenty, vaccination, coming, brexit,...",None
2,https://www.reddit.com/r/unitedkingdom/comment...,"If you don't listen to your core voter base, t...",1_11_1_1,"[n't, listen, core, voter, base, ,, wo, n't, v...",None
3,https://www.reddit.com/r/unitedkingdom/comment...,Much as I agree with your potential projection...,1_15,"[much, agree, potential, projection, ,, overwh...",None
4,https://www.reddit.com/r/unitedkingdom/comment...,"Brexit, the word, is why it succeeded. Nobody ...",2,"[brexit, ,, word, ,, succeeded, ., nobody, wou...",None
...,...,...,...,...,...
11709,https://www.reddit.com/r/unitedkingdom/comment...,"By Britain they don't mean ordinary people, mo...",20_1_1,"[britain, n't, mean, ordinary, people, ,, n't,...",None
11710,https://www.reddit.com/r/unitedkingdom/comment...,Because the people on the leave side see this ...,22,"[people, leave, side, see, establishment, find...",None
11711,https://www.reddit.com/r/unitedkingdom/comment...,Because ultimately it is the houses of parliam...,24,"[ultimately, house, parliament, voted, brexit,...",None
11712,https://www.reddit.com/r/unitedkingdom/comment...,"Democracy changes laws and governments, but po...",35,"[democracy, change, law, government, ,, politi...",None


# Using DistilBERT Model

In [1]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [13]:
df_reddit_DistilBERT = df_reddit.head(10).copy()

def get_score_DistilBERT (comment):
    encoded_input = tokenizer(comment, return_tensors='pt')
    output = model(**encoded_input)
    sentiment_score = output.logits.detach().numpy()[0]
    return sentiment_score

In [14]:
df_reddit_DistilBERT['score'] = df_reddit_DistilBERT['comment'].apply(get_score_DistilBERT)
df_reddit_DistilBERT

,url,comment,comment_id,score
0,https://www.reddit.com/r/unitedkingdom/comment...,"Agree. With the Tories pushing Brexit, they h...",1_2_1_4_1,"[-0.14808065, -0.17506288]"
1,https://www.reddit.com/r/unitedkingdom/comment...,I just thought that with plenty of vaccination...,1_8_1_1,"[-0.15040019, -0.16356972]"
2,https://www.reddit.com/r/unitedkingdom/comment...,"If you don't listen to your core voter base, t...",1_11_1_1,"[-0.14527896, -0.19329615]"
3,https://www.reddit.com/r/unitedkingdom/comment...,Much as I agree with your potential projection...,1_15,"[-0.12925428, -0.2017273]"
4,https://www.reddit.com/r/unitedkingdom/comment...,"Brexit, the word, is why it succeeded. Nobody ...",2,"[-0.10443308, -0.20493583]"
5,https://www.reddit.com/r/unitedkingdom/comment...,"""Brexit means Brexit""... =""\n\nThe one stand o...",2_1_3,"[-0.12687565, -0.19886912]"
6,https://www.reddit.com/r/unitedkingdom/comment...,The word actually wasnt really used much anyw...,2_2,"[-0.1233595, -0.21118498]"
7,https://www.reddit.com/r/unitedkingdom/comment...,"It was used, but only as a kind of novelty ter...",2_2_1_1_1_1,"[-0.1240364, -0.23696531]"
8,https://www.reddit.com/r/unitedkingdom/comment...,This is exact right - the portmanteau of Grex...,2_2_1_1_1_1_1,"[-0.10308735, -0.21843742]"
9,https://www.reddit.com/r/unitedkingdom/comment...,Its probably somewhere between the two of us ...,2_2_1_1_1_2,"[-0.16160363, -0.21258013]"


In [26]:
text = "I think life is amazing and everything is great."
#text = "I think Brexit is stupid and it should not have happened."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
sentiment_score = output.logits.detach().numpy()[0]
sentiment_score

array([-0.12553023, -0.1777198 ], dtype=float32)

# Using DistilBERT Model (Cosine Similarity)

In [23]:
df_reddit_DistilBERT = df_reddit.copy()

In [24]:
def get_reference_id(num):
    try:
        num_list = num.split("_")
    
        if len(num_list) > 1:
            num_list.pop()

        concat_num = '_'.join(num_list)   
        return concat_num
    except: 
        return 'NA'
    
def get_reference(url, reference_id):
    try:
        index = df_reddit.index[(df_reddit['comment_id'] == reference_id) & (df_reddit['url'] == url)]
        index_int = index.values.tolist()[0]
        reference_comment = df_reddit['comment'][index_int]
        return reference_comment        
    except:
        return 'NA'

In [33]:
pd.reset_option("display.max_colwidth")

In [ ]:
df_reddit_DistilBERT['reference_id'] = df_reddit_DistilBERT['comment_id'].apply(get_reference_id)
df_reddit_DistilBERT['reference_comment'] = df_reddit_DistilBERT.apply(lambda row: get_reference(row['url'], row['reference_id']), axis = 1)
df_reddit_DistilBERT = df_reddit_DistilBERT.reindex(columns=['url', 'comment', 'comment_id', 'reference_comment', 'reference_id'])

In [51]:
df_reddit_DistilBERT

,url,comment,comment_id,reference_comment,reference_id
0,https://www.reddit.com/r/unitedkingdom/comment...,"Agree. With the Tories pushing Brexit, they h...",1_2_1_4_1,NA,1_2_1_4
1,https://www.reddit.com/r/unitedkingdom/comment...,I just thought that with plenty of vaccination...,1_8_1_1,NA,1_8_1
2,https://www.reddit.com/r/unitedkingdom/comment...,"If you don't listen to your core voter base, t...",1_11_1_1,NA,1_11_1
3,https://www.reddit.com/r/unitedkingdom/comment...,Much as I agree with your potential projection...,1_15,NA,1
4,https://www.reddit.com/r/unitedkingdom/comment...,"Brexit, the word, is why it succeeded. Nobody ...",2,"Brexit, the word, is why it succeeded. Nobody ...",2
...,...,...,...,...,...
11709,https://www.reddit.com/r/unitedkingdom/comment...,"By Britain they don't mean ordinary people, mo...",20_1_1,NA,20_1
11710,https://www.reddit.com/r/unitedkingdom/comment...,Because the people on the leave side see this ...,22,Because the people on the leave side see this ...,22
11711,https://www.reddit.com/r/unitedkingdom/comment...,Because ultimately it is the houses of parliam...,24,Because ultimately it is the houses of parliam...,24
11712,https://www.reddit.com/r/unitedkingdom/comment...,"Democracy changes laws and governments, but po...",35,"Democracy changes laws and governments, but po...",35


In [14]:
from transformers import DistilBertTokenizer, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

statement1 = "The sky is blue."
statement2 = "The sky is green."

encoded_statement1 = tokenizer.encode(statement1, return_tensors='pt')
encoded_statement2 = tokenizer.encode(statement2, return_tensors='pt')

with torch.no_grad():
    vector1 = model(encoded_statement1)[0][:, 0, :].numpy()
    vector2 = model(encoded_statement2)[0][:, 0, :].numpy()

from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vector1, vector2)
print(similarity)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[[0.99577045]]
